# Error handling

ironArray has a sophisticated error handling procedure that allows us to catch errors from both
the C library and the Python library.

To see how all this machinery works we are going to create a very simple array and deal with some
invalid copies:

In [1]:
import iarray as ia
import numpy as np

In [2]:
dtshape = ia.DTShape((1000, 1000), dtype=np.float32)
a = ia.linspace(dtshape, 0, 1)

In [3]:
with ia.config(chunkshape=(100, 100), blockshape=(50, 50)) as cfg:
    a.copy(cfg=cfg)


As we can see, this first copy was created correctly.
But what happens if we create a copy with a chunkshape bigger than the shape of the array?

In [4]:
with ia.config(chunkshape=(10000, 10000), blockshape=(1000, 1000)) as cfg:
    a.copy(cfg=cfg)


IArrayError: b'CHUNK SHAPE NOT VALID - 0x800a000000938404 - error=1,ver=0,rev=10,os=0,neg=1,adj=147,subject=1028,code=9633792,ubits=0x0'

Now the ironArray machinery has returned us a Python error defined in ironArray called
``IArrayError``. This error simply returns a very general error message. In this particular case,
the error is ``CHUNK SHAPE NOT VALID`` but it does not indicate why it is not valid.

In order to obtain more detailed information about the error, we can set the environment variable
``INAC_TRACE="iarray.error"``. With this environment variable activated, ironArray will show us
a more detailed error and, in addition, it will provide us with a tracing of the C library.

To use this functionality inside a jupyter notebook (as in our case), we also have to use the
``wurlitzer`` extension to correctly handle stdout and stderr files.

In [5]:
%load_ext wurlitzer

import os
os.environ['INAC_TRACE'] = 'iarray.error'


In [6]:
with ia.config(chunkshape=(10000, 10000), blockshape=(1000, 1000)) as cfg:
    a.copy(cfg=cfg)

[iarray.error] - The chunkshape is bigger than shape /Users/aleix11alcacer/Projects/iron-array/iron-array-python/iarray/iarray-c-develop/src/iarray_container.c:271
[iarray.error] - Tracing:  /Users/aleix11alcacer/Projects/iron-array/iron-array-python/iarray/iarray-c-develop/src/iarray_constructor.c:458


IArrayError: b'CHUNK SHAPE NOT VALID - 0x800a000000938404 - error=1,ver=0,rev=10,os=0,neg=1,adj=147,subject=1028,code=9633792,ubits=0x0'

On the one hand, with the INAC_TRACE environment variable set we get the error message
``The chunkshape is bigger than shape`` that as we can see, this message
already gives us an idea of what we are doing wrong.

On the other hand, it also tells us in which lines of the code of the C library the error has
occurred. In this case:
- ``iarray/iarray-c-develop/src/iarray_container.c:271``
- ``iarray/iarray-c-develop/src/iarray_constructor.c:458``